In [1]:
from mygrad.nnet.layers import dense, RecurrentUnit 
from mygrad.nnet.activations import relu
from mygrad import Tensor


In [2]:
import numpy as np
import csv

In [3]:
def unzip(l):
    return zip(*l)

In [4]:
#Gets data from bitfinex exchange
with open("./bitstampUSD.csv", mode='r') as q:
    data=[]
    File=csv.reader(q, delimiter=',')
    for line in File:
        data.append(line)

        
 


In [5]:
print(data[-1])

['1500330220', '2175.000000000000', '0.330997170000']


In [6]:
timePriceVolume=list(unzip(data))


In [7]:
time=timePriceVolume[0]
price=timePriceVolume[1]
volume=timePriceVolume[2]

In [8]:
time=np.array(time,dtype=int)
price=np.array(price,dtype=float)
volume=np.array(volume,dtype=float)

In [9]:
def get_average_price(time,price,volume):
    tprice=np.reshape(price[3:],(2489638,5))
    tvol=np.reshape(volume[3:],(2489638,5))
    voltot=np.sum(tvol,1)
    tTime=time[3::5]
    weightedPrice=np.einsum('ij,ij->i', tprice, tvol)/voltot
    return (weightedPrice, tTime)
    
    

In [10]:
aPrice,Time=get_average_price(time,price,volume)

/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:6: RuntimeWarning: invalid value encountered in true_divide
  


In [11]:
print(aPrice[:20])
print(price[:100])

timeData=Time[-250000]
aPriceData=aPrice[-250000:]
price=0
volume=0
time=0
data=0

[ 5.95082231  5.72382026  5.56111985  5.62485348  5.13549632  5.06346123
  4.86593322  4.82371087  4.90651132  5.46313716  5.5284818   5.50676045
  5.70607488  5.69696046  5.70602647  5.73037783  5.75644634  5.79682393
  6.06        5.94585946]
[ 5.8   5.83  5.9   6.    5.95  5.88  5.88  5.76  5.65  5.92  5.95  5.97
  5.58  5.54  5.62  5.57  5.55  5.56  5.57  5.61  5.65  5.72  5.61  5.52
  5.52  5.53  5.12  5.02  5.    5.21  5.24  5.13  4.82  4.8   4.87  4.85
  4.87  4.87  4.82  4.81  4.85  4.81  4.82  4.82  4.92  4.9   4.92  4.93
  5.37  4.95  5.38  5.49  5.56  5.5   5.55  5.55  5.55  5.55  5.42  5.43
  5.56  5.57  5.66  5.7   5.79  5.69  5.79  5.66  5.68  5.68  5.69  5.7
  5.72  5.7   5.71  5.71  5.72  5.7   5.7   5.71  5.72  5.73  5.74  5.75
  5.75  5.75  5.76  5.76  5.7   5.68  5.68  6.05  6.05  6.06  6.06  6.06
  6.06  6.06  6.05  6.05]


In [12]:
print(aPriceData.shape)

(250000,)


In [13]:
x_train=aPriceData[:195000]
x_val=aPriceData[195000:200000]
x_test=aPriceData[200000:]


In [14]:
print(x_train.shape)
print(x_val.shape)
print(x_test.shape)

(195000,)
(5000,)
(50000,)


In [15]:
print(x_train[:100])

[ 1573.63168133  1574.31161724  1574.74836377  1574.78607643  1574.87149837
  1577.024869    1579.59509828  1579.90026608  1583.44135856  1584.84092204
  1583.94885239  1583.60917372  1582.9724752   1584.25022301  1584.86385779
  1582.13038605  1584.48463683  1584.02050217  1583.16914723  1582.90922226
  1583.9957872   1583.97946903  1582.26457072  1580.8         1580.52943056
  1578.02283221  1579.6933264   1575.74776494  1575.41802513  1574.92437604
  1575.16328929  1575.11631387  1575.2085594   1574.805       1572.63341422
  1573.3627997   1570.64912148  1570.59940833  1573.21936711  1571.89792379
  1573.7942594   1574.11598742  1569.91906152  1570.89626233  1572.19596914
  1570.35839768  1568.92396668  1568.43388743  1569.41880858  1568.43
  1568.22499375  1567.18716071  1567.21371197  1566.94226154  1565.84012391
  1564.52586167  1566.16472277  1564.31994637  1563.79472123  1564.50660496
  1563.22463927  1564.90569123  1564.00232632  1563.165773    1563.
  1562.33079688  1562.06  

In [16]:
def loss(x,y_true):
    return (((x-y_true)**2).sum())/(len(x))
def grad_descent(t_params,rate):
    for w in t_params:
        w.data-=(w.grad*rate)
def get_reg(t_params,reg):
    tot=0
    for W in t_params:
        tot=tot + reg * (W**2).sum()

In [17]:
#Reshaping Arrays of consecutive data into clumps
xtrain=np.copy(x_train)
# xtrain=np.reshape(xtrain,(1950,100))
# np.random.shuffle(xtrain)
# xtrain=np.transpose(xtrain)



In [18]:
print(xtrain.shape)

(195000,)


In [19]:
def get_clump(xtrain,clumpSize,sentenceSize):
    clump=[]
    for i in range(clumpSize):
        r=np.random.randint(0,len(xtrain)-sentenceSize)
        clump.append(xtrain[r:r+sentenceSize])
    clump=np.array(clump)
    clump=np.transpose(clump)
    clump=clump[...,np.newaxis]
    return clump

In [20]:
c=get_clump(xtrain,200,100)

In [21]:
#initailizes hyperparameters
#where d determines the number of trainable parameters for 1st layer
D=10000
#where c is the context (1)
C=1 
#where P is the size of the prediction(output)
P=1
#where q determines the number of trainable parameters for 2nd layer
Q=10000
#where O is output size(1)
O=1
#Learning rate
rate=1e-7
#Regulation strength
reg=1e5
#Back Propigation Level
bp_lim=20
#sentence size
S=100
#clump size
N=200

In [22]:
#intitailizing trainable parameters

#DxD, where d determines the number of trainable parameters for 1st layer
W_0=Tensor(0.001 * np.random.randn(*(D,D)))

#CxD where c is the context (1)
U_0=Tensor(0.001 * np.random.randn(*(C,D)))

#DxP where P is the size of the prediction(output)
V_0=Tensor(0.001 * np.random.randn(*(D,P)))

#QxQ, where q determines the number of trainable parameters for 2nd layer
W_1=Tensor(0.001 * np.random.randn(*(Q,Q)))

#PxQ
U_1=Tensor(0.001 * np.random.randn(*(P,Q)))

#QxO where O is output size(1)
V_1=Tensor(0.001 * np.random.randn(*(Q,O)))

#,W_1,U_1,V_1
paramList=[W_0,U_0,V_0]

In [23]:
#initailizes reccurent networks
recurrent0=RecurrentUnit(U_0,W_0,bp_lim)
#recurrent1=RecurrentUnit(U_1,W_1,bp_lim)


In [ ]:
def saveParams():
    

In [ ]:
Loss=[]
for i in range(400):
    
    clump=get_clump(xtrain,N,S)
    HiddenDescriptor0=recurrent0(clump)
    layer1=dense(HiddenDescriptor0,V_0)
#     HiddenDescriptor1=recurrent1(layer1)
#     layer2=dense(HiddenDescriptor,V_1)
    L=(loss(layer2,np.transpose(clump))+get_reg(paramList,reg))
    L.backward()
    grad_descent(paramList,rate)
    L.null_gradients()
    Loss.append(L.data)
    print(i)
    print(L.data)


In [ ]:
n=np.arange(10,dtype=float).reshape((2,5))
tot=np.sum(n,1)
t=-np.arange(10,dtype=float).reshape((2,5))
print(n)
print(tot)
print(t)
print(np.einsum('ij,ij->i', n, t))

In [ ]:
a=np.arange(64).reshape((16,4))
print(a)
a=np.transpose(a)
print(a)
a=np.transpose(a)
print(a)

In [ ]:
a=np.full((3,4,5),2)
b=np.full((5,10),1)
print(np.dot(a,b).shape)